In [128]:
import pandas as pd
from scipy import stats
import datetime
import rpy2
import seaborn as sns
import numpy as np
import xarray as xr
from kshape.core import kshape, zscore
from tslearn.clustering import KShape
import matplotlib.pyplot as plt

## 0. Preprocessing

In [58]:
df = pd.read_csv("ecs_series_1_uid_encoded(1).csv")
datetime_object = datetime.datetime.strptime("9/27/18 16:00", "%m/%d/%y %H:%M")
df['Date'] = df["ds"].apply(lambda x:datetime.datetime.strptime(x, "%m/%d/%y %H:%M"))
df = df.drop(columns = ['ds','region_no_factor','iz_no_factor','product_factor','instance_type_name_factor','instance_type_family_factor'])

In [59]:
df = df.groupby(by=['uid','Date']).sum().reset_index()
df = df.pivot(columns = 'Date',index = 'uid',values = 'vcpu_net_delead')

In [60]:
df_recency =  pd.DataFrame(columns = df.columns)
df_monetary =  pd.DataFrame(columns = df.columns)
df_frequency = pd.DataFrame(columns = df.columns)

In [61]:
df_test = df.sample(2000)

In [62]:
df_test.to_csv('test.csv')

## 1. RFM-analysis

In [63]:
for index, row in df_test.iterrows():
    arr = []
    arr = list(row)
    rec_arr = []
    mon_arr = []
    freq_arr = []
    start_date = row.index[0]
    last_purchase = np.nan
    first_purchase = 0 #flag
    total_purchases = 0
    #RECENCY Customer did a explicit activity, such as renting cores or releasing cores.
    for j, entry in enumerate(arr):
        if (np.isnan(entry)):            
            if (first_purchase==0):
                rec_arr.append(row.index[j]-start_date)
            else:
                rec_arr.append(row.index[j]-last_purchase)
        else:
            #not nan
            total_purchases+=1
            first_purchase = 1
            rec_arr.append(0)
            last_purchase = row.index[j]
        #FREQUENCY it can be a cumulative average, the number of events divided by the number of days that have passed
        freq_arr.append(total_purchases/(j+1))
    df_recency.loc[index] = rec_arr
    df_frequency.loc[index]=freq_arr
    #MONETARY  Moving average monetary value
    current_servers = 0
    total_days_of_purchases = 0 
    for j,entry in enumerate(arr):
        if (np.isnan(entry)):
            total_days_of_purchases+=current_servers
        else:
            current_servers+=entry
            total_days_of_purchases+=current_servers
        mon_arr.append(total_days_of_purchases/(j+1))
    df_monetary.loc[index] = mon_arr
df_recency=df_recency.applymap( lambda x:0 if (type(x)==int) else x.days)

## 2. Clustering

In [90]:
ksh_monetary = KShape(n_clusters=4)
df_monetary['label'] = ksh_monetary.fit(df_monetary.values)
ksh_frequency = KShape(n_clusters=4)
df_frequency['label'] = ksh_frequency.fit(df_frequency.values)
ksh_recency = KShape(n_clusters=4)
df_recency['label'] = ksh_recency.fit(df_recency.values)

0.079 --> 0.080 --> 
0.092 --> 0.063 --> 0.062 --> 0.062 --> 0.062 --> 
Resumed because of empty cluster
0.178 --> 0.176 --> 0.174 --> 0.162 --> 0.154 --> 0.153 --> 0.150 --> 0.148 --> 0.148 --> 0.148 --> 0.148 --> 0.147 --> 0.147 --> 0.147 --> 0.147 --> 0.147 --> 0.147 --> 0.147 --> 0.147 --> 0.147 --> 0.146 --> 0.146 --> 0.146 --> 0.146 --> 0.146 --> 0.146 --> 0.146 --> 0.146 --> 0.146 --> 0.146 --> 0.145 --> 0.145 --> 0.145 --> 0.145 --> 0.145 --> 0.145 --> 0.145 --> 0.145 --> 0.145 --> 0.145 --> 0.145 --> 0.145 --> 0.145 --> 0.144 --> 0.144 --> 0.144 --> 0.144 --> 0.144 --> 0.144 --> 0.144 --> 0.144 --> 0.144 --> 0.144 --> 0.144 --> 0.144 --> 0.144 --> 0.144 --> 0.143 --> 0.143 --> 0.143 --> 0.143 --> 0.143 --> 0.143 --> 0.143 --> 0.143 --> 0.143 --> 0.143 --> 0.143 --> 0.143 --> 0.143 --> 0.143 --> 0.142 --> 0.142 --> 0.142 --> 0.142 --> 0.142 --> 0.142 --> 0.142 --> 0.142 --> 0.142 --> 0.142 --> 0.142 --> 0.142 --> 0.142 --> 0.142 --> 0.142 --> 0.142 --> 0.142 --> 0.142 --> 0.142

In [122]:
df_monetary['label'] =ksh_monetary.labels_
df_frequency['label'] =ksh_frequency.labels_
df_recency['label'] =ksh_recency.labels_

In [ ]:
df_centroids = pd.DataFrame(columns = df_recency.drop(columns = 'label').columns)
for i in range(ksh_recency.cluster_centers_.shape[0]):
    df_centroids.loc['Centroid_for_recency_'+str(i)]= ksh_recency.cluster_centers_[i].ravel()
    df_centroids.loc['Centroid_for_frequency_'+str(i)]= ksh_frequency.cluster_centers_[i].ravel()
    df_centroids.loc['Centroid_for_monetary_'+str(i)]= ksh_monetary.cluster_centers_[i].ravel()

In [159]:
df_monetary.to_csv("monetary.csv")
df_frequency.to_csv("frequency.csv")
df_recency.to_csv("recency.csv")
df_centroids.to_csv("cluster_centroids.csv")